## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


## Overview

Penetrating the movie business requires some understanding of the ropes. Microsoft has identified a gap and an investment opportunity in this sector but they need more information on creating movies and what type of films to create. This analysis will enhance the understanding of the movie studios that have produced some of the best movies and the amount of money they used in creating these movies. This will give Microsoft a better understadning of what they are supposed to do and how much they are supposed to invest in creating movies for their movies to be ranked among the best. Using the data available, we can see the ratings, budget and reviews given by people fro different movies. 

## Data Preparation

In [1]:
# Importing the relevant libraries
import pandas as pd
import numpy as np
import sqlite3
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
# read the data. The 'r' in the code will read any special characters
df_bom = pd.read_csv(r'C:\Users\Arnold Chovu\Desktop\Data_Science\Projects\dsc-phase-1-project-v2-4\ZippedData\bom_movie_gross.csv')
df_bom.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [9]:
df_rtreviews = pd.read_csv(r'C:\Users\Arnold Chovu\Desktop\Data_Science\Projects\dsc-phase-1-project-v2-4\ZippedData\rt_reviews_tsv', sep='\t', encoding='unicode_escape')
df_rtreviews.head()

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"


In [23]:
print(df_rtreviews['review'].isnull().sum())
print(df_rtreviews.shape)

5563
(54432, 8)


In [10]:
df_rtmovieinfo = pd.read_csv(r'C:\Users\Arnold Chovu\Desktop\Data_Science\Projects\dsc-phase-1-project-v2-4\ZippedData\rt.movie_info.tsv', sep='\t', encoding='unicode_escape')
df_rtmovieinfo.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


In [16]:
df_tmdbmovies = pd.read_csv(r'C:\Users\Arnold Chovu\Desktop\Data_Science\Projects\dsc-phase-1-project-v2-4\ZippedData\tmdb_movies.csv', index_col = 0)
df_tmdbmovies.head()

,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


In [18]:
df_tnmovie_budget = pd.read_csv(r'C:\Users\Arnold Chovu\Desktop\Data_Science\Projects\dsc-phase-1-project-v2-4\ZippedData\tn.movie_budgets.csv')
df_tnmovie_budget.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [25]:
# Create a cnnection to open the SQL file
conn = sqlite3.Connection(r'C:\Users\Arnold Chovu\Desktop\Data_Science\Projects\dsc-phase-1-project-v2-4\ZippedData\im.db')

In [28]:
# Checking for the table names
q = '''
SELECT name
FROM sqlite_schema
WHERE type = 'table'
;'''
pd.read_sql(q, conn)

,name
0,movie_basics
1,directors
2,known_for
3,movie_akas
4,movie_ratings
5,persons
6,principals
7,writers


In [29]:
q = '''
SELECT *
FROM movie_ratings
;'''
pd.read_sql(q, conn)

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21
...,...,...,...
73851,tt9805820,8.1,25
73852,tt9844256,7.5,24
73853,tt9851050,4.7,14
73854,tt9886934,7.0,5


## Data Understanding

The data used in this analysis is from IMDB and Box Office Mojo which provide the relevant information to understand the buisiness of movie creation. 

In [30]:
df_bom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


In [33]:
df_bom['year'].value_counts().sort_index()

2010    328
2011    399
2012    400
2013    350
2014    395
2015    450
2016    436
2017    321
2018    308
Name: year, dtype: int64

The data from Box Office Mojo has 3387 entries and 5 columns which assist in our understanding of the amount of money made by different movies between 2010-2018. 